In [ ]:
# impor library
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
# impor dataset
dataset = pd.read_csv('../integration-data/big-data.csv')

In [ ]:
# cek dataset
dataset.head(10)

In [ ]:
dataset.max()

In [ ]:
# ubah nilai jenisKelamin
dataset.jenisKelamin = dataset.jenisKelamin.replace(['Male', 'L'], '0')
dataset.jenisKelamin = dataset.jenisKelamin.replace(['Female', 'P'], '1')
dataset.jenisKelamin = dataset.jenisKelamin.astype('int64')

In [ ]:
# impor dataset who
who_haz = pd.read_csv('../raw-datasets/dataset-who-stunting.csv', delimiter=';', decimal=',')
who_whz = pd.read_csv('../raw-datasets/dataset-who-wasting.csv', delimiter=';', decimal=',')

In [ ]:
# membuat kolom baru yaitu stunting dan wasting
dataset['z_stunting'] = 0.0
dataset['z_wasting'] = 0.0

In [ ]:
# ubah gender pada tabel who untuk sama dengan di dataset
# who haz => height-for-age z-scores
who_haz.Gender = who_haz.Gender.replace('Male', '0')
who_haz.Gender = who_haz.Gender.replace('Female', '1')
# who wfh => weight-for-height z-scores
who_whz.Gender = who_whz.Gender.replace('Male', '0')
who_whz.Gender = who_whz.Gender.replace('Female', '1')
# ubah format gender menjadi nominal
who_haz.Gender = who_haz.Gender.astype('float64')
who_whz.Gender = who_whz.Gender.astype('float64')

In [ ]:
# cari nilai unik yang tidak terdapat pada tabel whz
# nilai unik berupa nilai yang tidak bernilai multiplikasi 0.5
unique_height = dataset.loc[dataset.tinggiBadan % 0.5 != 0, ['jenisKelamin', 'tinggiBadan']]
unique_height = unique_height.rename(columns={'jenisKelamin':'Gender', 'tinggiBadan':'Length'})
unique_height = unique_height.drop_duplicates()
# gabung data unik kedalam data whz
who_whz = pd.concat([who_whz, unique_height])
# sortir data whz dari gender lalu tinggi
who_whz = who_whz.sort_values(['Gender', 'Length'], ignore_index=True)

In [ ]:
# interpolasi nilai unik berdasar data whz
# proses interpolasi dibedakan gender
whz_0 = who_whz.loc[who_whz.Gender == 0].interpolate(method='linear', limit_area='inside')
whz_1 = who_whz.loc[who_whz.Gender == 1].interpolate(method='linear', limit_area='inside')
# gabung hasil interpolasi menjadi tabel whz baru
who_whz = pd.concat([whz_0, whz_1])
# sortir data whz menurut tinggi
who_whz = who_whz.sort_values(['Length', 'Gender'], ignore_index=True)

In [ ]:
# cari nilai z-score stunting
# loop seluruh dataset
for stunting_cursor in dataset.itertuples():
    # loop untuk mencari data yang tepat di tabel haz
    for who_cursor in  who_haz.itertuples():
        # cek persamaan jenisKelamin
        if who_cursor.Gender == stunting_cursor.jenisKelamin:
            # cek persamaan umur
            if who_cursor.Month == stunting_cursor.umur:
                l_score = who_cursor.L
                m_score = who_cursor.M
                s_score = who_cursor.S
                # hitung nilai z-score
                z_score = (((stunting_cursor.tinggiBadan / m_score)**l_score) - 1) / (l_score * s_score)
                # masukkan nilai dalam kolom z_stunting
                dataset.loc[dataset.index == stunting_cursor.Index, 'z_stunting'] = round(z_score, 3)
                continue

In [ ]:
# cari nilai z-score wasting
# loop seluruh dataset
for wasting_cursor in dataset.itertuples():
    # loop untuk mencari data yang tepat di tabel haz
    for who_cursor in  who_whz.itertuples():
        # cek persamaan jenisKelamin
        if who_cursor.Gender == wasting_cursor.jenisKelamin:
            # cek persamaan tinggi
            if who_cursor.Length == wasting_cursor.tinggiBadan:
                l_score = who_cursor.L
                m_score = who_cursor.M
                s_score = who_cursor.S
                # hitung nilai z-score
                z_score = (((wasting_cursor.beratBadan / m_score)**l_score) - 1) / (l_score * s_score)
                # masukkan nilai dalam kolom z_stunting
                dataset.loc[dataset.index == wasting_cursor.Index, 'z_wasting'] = round(z_score, 3)
                continue

In [ ]:
# membuat kolom baru label yang mengindikasi status data
dataset['label'] = 0
# tentukan nilai label
# 0:sehat, 1:stunting, 2:wasting, 3:stunting&wasting
for status in dataset.itertuples():
    # cek sehat
    if status.z_stunting > -2.0 and status.z_wasting > -2.0:
        dataset.loc[dataset.index == status.Index, 'label'] = 0
    # cek stunting
    elif status.z_stunting <= -2.0:
        # cek stunting & wasting
        if status.z_wasting <= -2.0:
            dataset.loc[dataset.index == status.Index, 'label'] = 3
        # jika wasting gagal, otomatis stunting
        else:
            dataset.loc[dataset.index == status.Index, 'label'] = 1
    # jika stunting gagal dan sehat gagal, otomatis wasting
    else:
        dataset.loc[dataset.index == status.Index, 'label'] = 2

In [ ]:
# hitung total persebaran
totalSehat = dataset.loc[dataset.label == 0, 'label'].count()
totalStunting = dataset.loc[dataset.label == 1, 'label'].count()
totalWasting = dataset.loc[dataset.label == 2, 'label'].count()
totalKeduanya = dataset.loc[dataset.label == 3, 'label'].count()

# beri label tiap data
labels = ['sehat', 'stunting', 'wasting', 'stunting & wasting']
# beri spasi antar data
explode = [0.05, 0.05, 0.05, 0.05]
# beri warna setiap data
colors = ['lightgray', 'lightgray', 'lightgray', 'lightgray']

# visualisasikan ke matplot piechart
fig, ax = plt.subplots()
ax = plt.pie([totalSehat, totalStunting, totalWasting, totalKeduanya], explode=explode, labels=labels, colors=colors, autopct='%.2f%%')
plt.show()

In [ ]:
# impor library
from sklearn.model_selection import train_test_split as sk_split
# tetapkan fitur dan label
fitur = dataset.loc[:, ['jenisKelamin', 'umur', 'beratBadan', 'tinggiBadan']]
label = dataset.loc[:, ['label']]
# bagi dataset menjadi 2 data, latih dan uji
latih_fitur, uji_fitur, latih_target, uji_target = sk_split(fitur, label, test_size=0.20)

In [ ]:
# hitung total persebaran
totalSehat = latih_target.loc[latih_target.label == 0, 'label'].count()
totalStunting = latih_target.loc[latih_target.label == 1, 'label'].count()
totalWasting = latih_target.loc[latih_target.label == 2, 'label'].count()
totalKeduanya = latih_target.loc[latih_target.label == 3, 'label'].count()

# beri label tiap data
labels = ['sehat', 'stunting', 'wasting', 'stunting & wasting']
# beri spasi antar data
explode = [0.05, 0.05, 0.05, 0.05]
# beri warna setiap data
colors = ['lightgray', 'lightgray', 'lightgray', 'lightgray']

# visualisasikan ke matplot piechart
fig, ax = plt.subplots()
ax = plt.pie([totalSehat, totalStunting, totalWasting, totalKeduanya], explode=explode, labels=labels, colors=colors, autopct='%.2f%%')
plt.show()

In [ ]:
# menerapkan oversampling dengan SMOTE
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=18062025)
latih_fitur_resample, latih_target_resample = sm.fit_resample(latih_fitur, latih_target)

In [ ]:
# hitung total persebaran
totalSehat = latih_target_resample.loc[latih_target_resample.label == 0, 'label'].count()
totalStunting = latih_target_resample.loc[latih_target_resample.label == 1, 'label'].count()
totalWasting = latih_target_resample.loc[latih_target_resample.label == 2, 'label'].count()
totalKeduanya = latih_target_resample.loc[latih_target_resample.label == 3, 'label'].count()

# beri label tiap data
labels = ['sehat', 'stunting', 'wasting', 'stunting & wasting']
# beri spasi antar data
explode = [0.05, 0.05, 0.05, 0.05]
# beri warna setiap data
colors = ['lightgray', 'lightgray', 'lightgray', 'lightgray']

# visualisasikan ke matplot piechart
fig, ax = plt.subplots()
ax = plt.pie([totalSehat, totalStunting, totalWasting, totalKeduanya], explode=explode, labels=labels, colors=colors, autopct='%.2f%%')
plt.show()

In [ ]:
# gabung data fitur dan label pada kedua dataset
dataset_latih = pd.concat([latih_fitur_resample, latih_target_resample], axis=1)
dataset_uji = pd.concat([uji_fitur, uji_target], axis=1)

In [ ]:
# ganti format umur menjadi float
dataset_latih.umur = dataset_latih.umur.astype('float64')
dataset_uji.umur = dataset_uji.umur.astype('float64')

In [ ]:
# normalisasi dataset latih
umur_min = dataset_latih.umur.min()
umur_max = dataset_latih.umur.max()
tinggi_min = dataset_latih.tinggiBadan.min()
tinggi_max = dataset_latih.tinggiBadan.max()
berat_min = dataset_latih.beratBadan.min()
berat_max = dataset_latih.beratBadan.max()
for cursor_data in dataset_latih.itertuples():
    current_umur = dataset_latih.loc[dataset_latih.index == cursor_data.Index, 'umur']
    dataset_latih.loc[dataset_latih.index == cursor_data.Index, 'umur'] = (current_umur - umur_min) / (umur_max - umur_min)
    current_tinggi = dataset_latih.loc[dataset_latih.index == cursor_data.Index, 'tinggiBadan']
    dataset_latih.loc[dataset_latih.index == cursor_data.Index, 'tinggiBadan'] = (current_tinggi - tinggi_min) / (tinggi_max - tinggi_min)
    current_berat = dataset_latih.loc[dataset_latih.index == cursor_data.Index, 'beratBadan']
    dataset_latih.loc[dataset_latih.index == cursor_data.Index, 'beratBadan'] = (current_berat - berat_min) / (berat_max - berat_min)

In [ ]:
# normalisasi dataset uji
umur_min = dataset_uji.umur.min()
umur_max = dataset_uji.umur.max()
tinggi_min = dataset_uji.tinggiBadan.min()
tinggi_max = dataset_uji.tinggiBadan.max()
berat_min = dataset_uji.beratBadan.min()
berat_max = dataset_uji.beratBadan.max()
for cursor_data in dataset_uji.itertuples():
    current_umur = dataset_uji.loc[dataset_uji.index == cursor_data.Index, 'umur']
    dataset_uji.loc[dataset_uji.index == cursor_data.Index, 'umur'] = (current_umur - umur_min) / (umur_max - umur_min)
    current_tinggi = dataset_uji.loc[dataset_uji.index == cursor_data.Index, 'tinggiBadan']
    dataset_uji.loc[dataset_uji.index == cursor_data.Index, 'tinggiBadan'] = (current_tinggi - tinggi_min) / (tinggi_max - tinggi_min)
    current_berat = dataset_uji.loc[dataset_uji.index == cursor_data.Index, 'beratBadan']
    dataset_uji.loc[dataset_uji.index == cursor_data.Index, 'beratBadan'] = (current_berat - berat_min) / (berat_max - berat_min)

In [ ]:
# acak urutan dataset
dataset_latih = dataset_latih.sample(frac=1, random_state=18062025, ignore_index=True)
dataset_uji = dataset_uji.sample(frac=1, random_state=18062025, ignore_index=True)

In [ ]:
# ekspor dataset
dataset_latih.to_csv('data_latih.csv', index=False)
dataset_uji.to_csv('data_uji.csv', index=False)